### Load Images

In [ ]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '0'
from visual_tokenizer.directsam import DirectSAMTokenizer

image_dir = '/home/dchenbs/workspace/datasets/coco2017/images/val2017'
image_paths = []
for root, dirs, files in os.walk(image_dir):
    for file in files:
        if file.endswith('.jpg') or file.endswith('.png'):
            image_paths.append(os.path.join(root, file))

print(len(image_paths))

In [ ]:
import torch
from model.utils import create_vlm
from model.utils import VisualTextualTokenization
from PIL import Image as PILImage

model, textual_tokenizer = create_vlm(
    llm = 'HuggingFaceTB/SmolLM-135M', 
    vlm_config = 'configs/vlm/debug.json', 
    model_max_length=1024
    )

model = model.cuda().half()

model.config.vm_loss_weight = 1
model.config.lm_loss_weight = 1
model.config.insert_queries = True

In [ ]:
visual_tokenizer = DirectSAMTokenizer(
    ckpt="chendelong/DirectSAM-tiny-distilled-30ep-plus-50ep-1024px-0910",
    threshold=0.1,
    image_resolution=1024,
    max_tokens=128,
    device="cuda"
)
vl_tokenizer = VisualTextualTokenization(textual_tokenizer, visual_tokenizer)

inputs = [
    {
        'image': PILImage.open(image_paths[0]),
        'text': '<|image|><|assistant|>a cat sitting on a table'
    }
]

inputs = vl_tokenizer(inputs)
print(inputs.keys())

In [ ]:
with torch.no_grad():
    outputs = model(**inputs)
    print(outputs.keys())
    print(outputs['loss'])

In [ ]:
# save model
model.save_pretrained('smollm')
textual_tokenizer.save_pretrained('smollm')
model, textual_tokenizer = create_vlm('smollm')

model = model.cuda().half()